## 讀取股票資料(使用 ```yfinance``` 或 ```yahoofinancials``` 或 ```Alpha Vintage```)

## 1. ```yfinance```:

在 ```anaconda``` 安裝 ```yfinance``` 套件: ```pip install yfinance --upgrade --no-cache-dir```   
註: 不知為何用 ```conda install -c ranaroussi yfinance``` 出現錯誤

### 讀取股票資料(```.download()```)

In [3]:
import yfinance as yf

In [4]:
data = yf.download('00850.TW', period='1y') # yfinance 支援 intra data，所以 period 可以小於一天

[*********************100%***********************]  1 of 1 completed


### 讀取多個股票的資料:

In [5]:
import datetime as dt
import pandas as pd

In [6]:
# 使用部分 0050 成分股為例:
stocks = ['1101.TW', '2207.TW', '2317.TW', '2454.TW', '2633.TW', '2880.TW', '2330.TW']

In [7]:
# 資料時間範圍是 360 天:
start =  dt.datetime.today() - dt.timedelta(360)
end = dt.datetime.today()

In [8]:
# 創建一個空的 data frame:
cl_price = pd.DataFrame() 

In [9]:
# 把每張股票的調整收盤價讀入 data frame:
for ticker in stocks:
    cl_price[ticker] = yf.download(ticker, start, end)['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


另一種做法: 剛才存成 data frame 的缺點是不能存放股票的所有資料，因此這次用 ```dictionary```:

In [10]:
# ohlcv stands for: Open, High, Low, Close, Volume
ohlcv_data = {}

In [11]:
for ticker in stocks:
    ohlcv_data[ticker] = yf.download(ticker, start, end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
ohlcv_data['2330.TW']['Low']

Date
2020-01-20    333.0
2020-01-30    316.5
2020-01-31    319.0
2020-02-03    312.0
2020-02-04    325.0
              ...  
2021-01-08    571.0
2021-01-11    574.0
2021-01-12    582.0
2021-01-13    593.0
2021-01-14    587.0
Name: Low, Length: 242, dtype: float64

### 2. ```yahoofinancials```:

這是另外一個 developer 寫的 library，是運用爬蟲技術  
```pip install yahoofinancials```

In [13]:
from yahoofinancials import YahooFinancials

In [14]:
ticker = '2317.TW'
yahoo_financials = YahooFinancials(ticker) # make a YahooFinancials object

In [15]:
data = yahoo_financials.get_historical_price_data("2019-01-13", "2020-01-14", "daily")

```get_history_price_data()``` 回傳的是 JSON 格式的資料  
```JSON``` 基本上就是巢狀的 ```dicrionary```，下一步是將 ```JSON``` 格式轉為 ```pandas``` 的 ```Data Frame```

In [16]:
close_prices = pd.DataFrame() 

In [17]:
end_date = (dt.datetime.today()).strftime("%Y-%m-%d") # .strftime() 將日期轉成 string。這邊要改成 YahooFinancials 接受的樣式
beg_date = (dt.datetime.today() - dt.timedelta(220)).strftime("%Y-%m-%d")
stocks = ['1101.TW', '2207.TW', '2317.TW', '2454.TW', '2633.TW', '2880.TW', '2330.TW']

### 目標: 將 ```json``` data 轉為 ```Data Frame```: 
<span style="color:blue"> 遇到的問題</font>: VS Code 裡的 data viewer 沒辦法顯示 ```json``` data(沒辦法知道 ```json_obj``` 的長相))。  
解決方法: 將資料存在電腦，直接觀看，做法:  
```python
import json
with open('stock.data', 'w', encoding='utf-8') as f:
        json.dump(json_obj, f) # .dump(): 將資料存成 json 檔
```

In [18]:
import json

In [19]:
for ticker in stocks:
    yahoo_financials = YahooFinancials(ticker)
    json_obj = yahoo_financials.get_historical_price_data(beg_date, end_date, "daily")
    
    ohlv = json_obj[ticker]['prices'] # ohlv 為 list
    temp = pd.DataFrame(ohlv)[['formatted_date', 'adjclose']] # 把 ohlv 的這兩個變數作為 temp 這個 data frame 的 column

    temp.set_index("formatted_date", inplace=True) # 將 temp 的 row index 設為 formatted_date 的資料，recall "inplace" 使改變可以永久套用在 temp 上
    temp.dropna(inplace=True) # 去掉沒有資料的部分
    #print(temp)

    close_prices[ticker] = temp["adjclose"] # 將 temp data frame(row 名稱跟資料都會一起搬過去) 資料套用在剛才的空 data frame  
print(close_prices)

                  1101.TW     2207.TW     2317.TW     2454.TW    2633.TW  \
formatted_date                                                             
2020-06-08      39.492817  612.320862   75.803139  485.623596  37.487782   
2020-06-09      39.447906  635.871643   75.612923  488.569763  37.245926   
2020-06-10      39.537731  671.197876   75.993370  491.024872  37.245926   
2020-06-11      38.908775  674.141724   74.756927  487.587708  36.762215   
2020-06-12      38.549290  665.310181   74.186264  483.659515  36.278500   
...                   ...         ...         ...         ...        ...   
2021-01-07      42.849998  634.000000  107.000000  799.000000  30.750000   
2021-01-08      42.950001  645.000000  108.000000  838.000000  30.700001   
2021-01-11      43.099998  639.000000  107.500000  851.000000  30.700001   
2021-01-12      42.700001  637.000000  104.000000  846.000000  30.650000   
2021-01-13      42.700001  649.000000  106.500000  884.000000  30.500000   

           

### 3. Alpha Vantage:

一樣是另一個抓股票的工具，但是這次需要 API，[到他們的網站](https://www.alphavantage.co/)免費下載 API:  
我的 API 為: ```21E8AYPJ0ZKQY7IO```，並將 API 存放於: ```C:\Users\User\Quant_Trading\alpha_vantage\api_key.txt```  
接下來可以安裝 alpha vantage 的套件(跟 Alpha Vantage 的公司不一樣，套件別人寫的): ```pip install alpha_vantage```

In [20]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

將 API 讀入，並把 TimeSeries 物件的 output 形式設為 ```pandas```:

In [25]:
key_path = r"C:\\Users\User\\Quant_Trading\\alpha_vantage\\api_key.txt"

In [26]:
ts = TimeSeries(key=open(key_path, 'r').read(), output_format="pandas")

找尋微軟的每日股票資訊，並將週期(```outputsize```)設為```full```，即所有期間的資料(另一個選項是 ```compact   ```)

In [32]:
data = ts.get_daily(symbol='MSFT', outputsize="full")
data

(            1. open  2. high    3. low  4. close   5. volume
 date                                                        
 2021-01-13   214.02   216.76  213.9266    216.34  20087080.0
 2021-01-12   216.50   217.10  213.3202    214.93  23148341.0
 2021-01-11   218.47   218.91  216.7300    217.49  23047029.0
 2021-01-08   218.68   220.58  217.0261    219.62  22956206.0
 2021-01-07   214.04   219.34  213.7100    218.29  27694480.0
 ...             ...      ...       ...       ...         ...
 1999-11-05    91.81    92.87   90.5000     91.56  35083700.0
 1999-11-04    92.31    92.75   90.3100     91.75  27119700.0
 1999-11-03    92.94    93.50   91.5000     92.00  22258500.0
 1999-11-02    92.75    94.50   91.9400     92.56  23174500.0
 1999-11-01    93.25    94.19   92.1200     92.37  26630600.0
 
 [5335 rows x 5 columns],
 {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'MSFT',
  '3. Last Refreshed': '2021-01-13',
  '4. Output Size': 'Full size',
